## Stability

In [1]:
# Set root directory of the project as the current working directory
import os
initial_dir = os.getcwd()  # Save initial directory (notebooks/)
os.chdir('..')  # Move to project/

# Other imports
import numpy as np
import shap

import logging
logging.basicConfig(level=logging.WARNING)

# Set the random seed for reproducibility
np.random.seed(42)

# Import Config from config.defaults and load_preprocessed_data, train_lstm_model, train_cnn_model from src.models
from config.defaults import Config
from src.models import load_saved_model, load_preprocessed_data
from src.stability import (calculate_relative_input_stability, 
                            calculate_relative_representation_stability, 
                            calculate_relative_output_stability)

# Load the default configuration
config = Config()
model_task = 'lstm_regression'

# Load preprocessed data
X_train, X_val, X_test, y_train_reg, y_val_reg, y_test_reg, metadata = load_preprocessed_data(model_task = model_task, eol_capacity=config.eol_capacity)

# Load saved LSTM model
lstm_model = load_saved_model(model_task, config)

# Define the predict function for SHAP
f = lambda x: lstm_model.predict(x)

In [3]:
# Sample 50 random sequences from the train set
X_train_2d = X_train.reshape(X_train.shape[0], -1)
X_background = shap.sample(X_train_2d, 50)

# Reshape the test set to 2D
X_test_2d = X_test.reshape(X_test.shape[0], -1)

# extract one sequence from the test set
test_instance = shap.sample(X_test_2d, 1)

In [4]:
# RIS
ris_max, ris_mean = calculate_relative_input_stability(
    lstm_model, X_background, test_instance, noise_scale=0.01
)
print(f"RIS - Max: {ris_max:.4f}, Mean: {ris_mean:.4f}")

/opt/miniconda3/envs/D2D_env/lib/python3.11/site-packages/sklearn/linear_model/_least_angle.py:755: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 95 iterations, alpha=2.586e-05, previous alpha=2.585e-05, with an active set of 52 regressors.
  warnings.warn(


RIS - Max: 2.1849, Mean: 1.5053


In [7]:
# After loading the model
lstm_model = load_saved_model(model_task, config)

# Warm up the model with a dummy input (use test_instance reshaped to 3D)
dummy_input = test_instance.reshape(1, test_instance.shape[1], 1)  # Shape: (1, timesteps, 1)
lstm_model.predict(dummy_input, verbose=0)  # This builds the graph

array([[0.5622587]], dtype=float32)

In [9]:
# Check layer existence
try:
    lstm_layer = lstm_model.get_layer("lstm_1")
    print("Layer 'lstm_1' found:", lstm_layer)
except ValueError as e:
    print("Layer 'lstm_1' not found:", e)

Layer 'lstm_1' found: <LSTM name=lstm_1, built=True>


In [8]:
# RRS
rrs_max, rrs_mean = calculate_relative_representation_stability(
    lstm_model, test_instance, noise_scale=0.01, layer_name="lstm_1"
)
print(f"RRS - Max: {rrs_max:.4f}, Mean: {rrs_mean:.4f}")

AttributeError: The layer sequential_1 has never been called and thus has no defined input.

In [6]:
# ROS
ros_max, ros_mean = calculate_relative_output_stability(
    lstm_model, test_instance, noise_scale=0.01
)
print(f"ROS - Max: {ros_max:.4f}, Mean: {ros_mean:.4f}")

ROS - Max: 1.8245, Mean: 0.7277
